# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# testing the conda environment, this is not related to the project
import sys
print(sys.executable)  

/opt/miniconda3/envs/rnn/bin/python


In [2]:
# imports
import pandas as pd
from transformers import pipeline

### Get data from `.csv` files and then preprocess data

In [11]:
def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files with consistent column names and concatenates them 
    into one resultant dataframe with 'Original Language' column added.
    """
    # Read CSV files with consistent column names
    column_names = ['Title', 'Year', 'Synopsis', 'Review']
    
    df_eng = pd.read_csv("data/movie_reviews_en.csv", names=column_names, header=0)
    df_fr = pd.read_csv("data/movie_reviews_fr.csv", names=column_names, header=0)
    df_sp = pd.read_csv("data/movie_reviews_sp.csv", names=column_names, header=0)
    
    df_eng['Original Language'] = 'en'
    df_fr['Original Language'] = 'fr'
    df_sp['Original Language'] = 'sp'
    
    df_combined = pd.concat([df_eng, df_fr, df_sp], ignore_index=True)
    
    return df_combined

df = preprocess_data()

In [12]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",en
29,El Incidente,2014,"En esta película de terror mexicana, un grupo ...","""El Incidente es una película aburrida y sin s...",sp
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",en
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",en
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",en
18,Les Visiteurs en Amérique,2000,Dans cette suite de la comédie française Les V...,"""Le film est une perte de temps totale. Les bl...",fr
20,Roma,2018,Cleo (Yalitza Aparicio) es una joven empleada ...,"""Roma es una película hermosa y conmovedora qu...",sp
16,La Tour Montparnasse Infernale,2001,Deux employés de bureau incompétents se retrou...,"""Je ne peux pas croire que j'ai perdu du temps...",fr
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",en
12,Amélie,2001,Cette comédie romantique raconte l'histoire d'...,"""Amélie est un film absolument charmant qui vo...",fr


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [13]:
from transformers import MarianMTModel, MarianTokenizer

fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"

fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)

fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

def translate(text: str, model, tokenizer) -> str:
    """
    Function to translate text using a model and tokenizer
    
    Parameters:
        text (str): Single text string to translate
        model: The translation model
        tokenizer: The tokenizer for the model
    Returns:
        str: Translated text
    """
    if not isinstance(text, str):
        text = str(text)
        
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model.generate(**inputs)
    translated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return translated


/opt/miniconda3/envs/rnn/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [14]:
fr_mask = df['Original Language'] == 'fr'
sp_mask = df['Original Language'] == 'sp'

#  previous version that I have changed 
'''
for idx in df[fr_mask].index:
    df.at[idx, 'Review'] = translate(df.at[idx, 'Review'], fr_en_model, fr_en_tokenizer)
    df.at[idx, 'Synopsis'] = translate(df.at[idx, 'Synopsis'], fr_en_model, fr_en_tokenizer)

for idx in df[sp_mask].index:
    # Translate review
    df.at[idx, 'Review'] = translate(df.at[idx, 'Review'], es_en_model, es_en_tokenizer)
    # Translate synopsis
    df.at[idx, 'Synopsis'] = translate(df.at[idx, 'Synopsis'], es_en_model, es_en_tokenizer)
'''

translation_configs = {
    'fr': (fr_mask, fr_en_model, fr_en_tokenizer),
    'sp': (sp_mask, es_en_model, es_en_tokenizer)
}

for lang, (mask, model, tokenizer) in translation_configs.items():
    for col in ['Review', 'Synopsis']:
        df.loc[mask, col] = df.loc[mask, col].apply(
            lambda x: translate(x, model, tokenizer)
        )

In [15]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
21,La Casa de Papel,(2017-2021),This Spanish television series follows a group...,"""The Paper House is an exciting and addictive ...",sp
11,Intouchables,2011,This film tells the story of the unlikely frie...,"""Untouchables is an incredibly touching film w...",fr
20,Roma,2018,Cleo (Yalitza Aparicio) is a young domestic wo...,"""Rome is a beautiful and moving film that pays...",sp
15,Le Dîner de Cons,1998,The film follows the story of a group of rich ...,"""I didn't like this movie at all. The concept ...",fr
13,Les Choristes,2004,This film tells the story of a music teacher w...,"""The Choristes are a beautiful film that will ...",fr
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",en
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",en
24,Amores perros,2000,Three stories intertwine in this Mexican film:...,"""Amores dogs is an intense and moving film tha...",sp
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",en
22,Y tu mamá también,2001,Two teenage friends (Gael García Bernal and Di...,"""And your mom is also a movie that stays with ...",sp


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [17]:
import torch

# MPS for Apple machine 
device = (
    "mps" if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available()
    else "cpu"
)

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_classifier = pipeline(
    "sentiment-analysis", 
    model=model_name,
    device=device 
)

def analyze_sentiment(text, classifier):
    """
    Function to perform sentiment analysis on text
    """
    result = classifier(text)[0]
    # Return "Positive" if POSITIVE label and score > 0.5, else "Negative"
    return "Positive" if result['label'] == 'POSITIVE' and result['score'] > 0.5 else "Negative"


In [18]:
def clean_text(text):
    """Clean and validate text data"""
    if pd.isna(text) or text == 'nan' or not isinstance(text, str):
        return ""
    return text.strip('"')  # Remove quote marks that appear in reviews

import os
os.makedirs("result", exist_ok=True)

df['Review'] = df['Review'].apply(clean_text)
df = df[df['Review'] != ""]

df['Review Sentiment'] = df['Review'].apply(lambda x: analyze_sentiment(x, sentiment_classifier))

df = df[['Title', 'Year', 'Synopsis', 'Review', 'Review Sentiment', 'Original Language']]

df = df.head(30)

if len(df) != 30:
    print(f"Warning: Expected 30 rows but got {len(df)} rows")

required_columns = ['Title', 'Year', 'Synopsis', 'Review', 'Review Sentiment', 'Original Language']
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    print(f"Warning: Missing required columns: {missing_columns}")

# Save to CSV
df.to_csv("result/reviews_with_sentiment.csv", index=False)

In [19]:
df.sample(10)

,Title,Year,Synopsis,Review,Review Sentiment,Original Language
10,La La Land,2016,This musical tells the story of a budding actr...,The Land is an absolutely beautiful film with ...,Positive,fr
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,It was difficult to sit through the whole thin...,Negative,en
20,Roma,2018,Cleo (Yalitza Aparicio) is a young domestic wo...,Rome is a beautiful and moving film that pays ...,Positive,sp
18,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,The film is a total waste of time. The jokes a...,Negative,fr
16,La Tour Montparnasse Infernale,2001,Two incompetent office workers find themselves...,I can't believe I've wasted time watching this...,Negative,fr
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","The Nice Guys tries too hard to be funny, and ...",Negative,en
11,Intouchables,2011,This film tells the story of the unlikely frie...,Untouchables is an incredibly touching film wi...,Positive,fr
13,Les Choristes,2004,This film tells the story of a music teacher w...,The Choristes are a beautiful film that will m...,Positive,fr
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,The Godfather is a classic movie that stands t...,Positive,en
14,Le Fabuleux Destin d'Amélie Poulain,2001,This romantic comedy tells the story of Amélie...,The Fabulous Destiny of Amélie Poulain is an a...,Positive,fr


In [21]:
df.to_csv("result/reviews_with_sentiment.csv", index=False)